In [1]:
!pip install tensorflow==2.13.0 --force-reinstall
!pip install tflite-support

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully uninstalled protobuf-4.25.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.13.0 which is incompatible.


In [1]:
from tflite_support import metadata_schema_py_generated as _metadata_fb
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tflite_support.metadata_writers import writer_utils
from sklearn.model_selection import train_test_split
from tflite_support import metadata as _metadata
from tflite_support import metadata_writers
from tflite_support import metadata
import flatbuffers
import tensorflow as tf
import numpy as np
import zipfile
import shutil
import os
import io


In [2]:
!gdown --id 17qjW3qhjaWSlLZvjMlbeTVi9y7Nx1N9X

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=17qjW3qhjaWSlLZvjMlbeTVi9y7Nx1N9X
From (redirected): https://drive.google.com/uc?id=17qjW3qhjaWSlLZvjMlbeTVi9y7Nx1N9X&confirm=t&uuid=ef91a0ca-fcc4-486b-bbd5-09aacea29293
To: /content/dataset.zip
100% 81.0M/81.0M [00:01<00:00, 64.3MB/s]


In [3]:
current_dir = os.getcwd()
local_zip = './dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()
zip_ref.close()

In [4]:
# Set the dataset folder path
dataset_folder = 'dataset'

# Get the list of class folders
class_folders = [f for f in os.listdir(dataset_folder) if os.path.isdir(os.path.join(dataset_folder, f))]

# Create lists to store the image file paths and their corresponding labels
image_files = []
labels = []

# Iterate through each class folder
for class_folder in class_folders:
    class_folder_path = os.path.join(dataset_folder, class_folder)
    image_files_in_class = [os.path.join(class_folder_path, f) for f in os.listdir(class_folder_path) if f.endswith('.jpg')]
    image_files.extend(image_files_in_class)
    labels.extend([class_folder] * len(image_files_in_class))

# Convert the lists to numpy arrays
image_files = np.array(image_files)
labels = np.array(labels)

# Split the data into training and testing sets
train_files, test_files, train_labels, test_labels = train_test_split(image_files, labels, test_size=0.3, random_state=42, stratify=labels)

# Create train and test folders if they don't exist
if not os.path.exists(os.path.join(dataset_folder, 'train')):
    os.makedirs(os.path.join(dataset_folder, 'train'))
if not os.path.exists(os.path.join(dataset_folder, 'test')):
    os.makedirs(os.path.join(dataset_folder, 'test'))

# Move images to train and test folders
for file in train_files:
    class_folder = os.path.basename(os.path.dirname(file))
    if not os.path.exists(os.path.join(dataset_folder, 'train', class_folder)):
        os.makedirs(os.path.join(dataset_folder, 'train', class_folder))
    os.rename(file, os.path.join(dataset_folder, 'train', class_folder, os.path.basename(file)))

for file in test_files:
    class_folder = os.path.basename(os.path.dirname(file))
    if not os.path.exists(os.path.join(dataset_folder, 'test', class_folder)):
        os.makedirs(os.path.join(dataset_folder, 'test', class_folder))
    os.rename(file, os.path.join(dataset_folder, 'test', class_folder, os.path.basename(file)))

In [5]:
train_dir = "/content/dataset/train"
test_dir = "/content/dataset/test"

train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(229, 229),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(229, 229),
    batch_size=32,
    class_mode='categorical'
)

Found 339 images belonging to 64 classes.
Found 146 images belonging to 64 classes.


In [6]:
classes_train = train_generator.class_indices
classes_test = test_generator.class_indices
print("Training samples:", train_generator.samples)
print("Testing samples:", test_generator.samples)

Training samples: 339
Testing samples: 146


In [7]:
sorted_class_indices = sorted(train_generator.class_indices.items(), key=lambda x: x[1])
sorted_class_names = [class_name for class_name, index in sorted_class_indices]

labels_path = "labels.txt"

with open(labels_path, 'w') as f:
    for class_name in sorted_class_names:
        f.write(class_name + '\n')

print(f"Labels saved to {labels_path}")

Labels saved to labels.txt


In [8]:
# Paths for the model and label file
_MODEL_PATH = "mobilenetv2_model.tflite"
_LABEL_FILE = "labels.txt"
_SAVE_TO_PATH = "mobilenetv2_model_metadata.tflite"

# Normalization parameters for the input image
_INPUT_NORM_MEAN = 127.5
_INPUT_NORM_STD = 127.5

# Create model metadata
model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = "MobileNetV2 Gizilo image classifier"
model_meta.description = (
    "Identify branded retail products and make grading for these products "
    "based on the calculation of nutritional value."
)
model_meta.version = "v1"
model_meta.author = "Core Gizilo"
model_meta.license = (
    "Apache License. Version 2.0 "
    "http://www.apache.org/licenses/LICENSE-2.0."
)

# Create input metadata
input_meta = _metadata_fb.TensorMetadataT()
input_meta.name = "image"
input_meta.description = (
    "Input image to be classified. The expected image is 229 x 229, with "
    "three channels (red, blue, and green) per pixel. Each value in the "
    "tensor is a single byte between 0 and 255."
)
input_meta.content = _metadata_fb.ContentT()
input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
input_meta.content.contentProperties.colorSpace = _metadata_fb.ColorSpaceType.RGB
input_meta.content.contentPropertiesType = _metadata_fb.ContentProperties.ImageProperties

input_normalization = _metadata_fb.ProcessUnitT()
input_normalization.optionsType = _metadata_fb.ProcessUnitOptions.NormalizationOptions
input_normalization.options = _metadata_fb.NormalizationOptionsT()
input_normalization.options.mean = [_INPUT_NORM_MEAN]
input_normalization.options.std = [_INPUT_NORM_STD]
input_meta.processUnits = [input_normalization]

input_stats = _metadata_fb.StatsT()
input_stats.max = [255]
input_stats.min = [0]
input_meta.stats = input_stats

# Create output metadata
output_meta = _metadata_fb.TensorMetadataT()
output_meta.name = "probability"
output_meta.description = "Probabilities of the labels respectively."
output_meta.content = _metadata_fb.ContentT()
output_meta.content.contentProperties = _metadata_fb.FeaturePropertiesT()
output_meta.content.contentPropertiesType = _metadata_fb.ContentProperties.FeatureProperties

output_stats = _metadata_fb.StatsT()
output_stats.max = [1.0]
output_stats.min = [0.0]
output_meta.stats = output_stats

label_file = _metadata_fb.AssociatedFileT()
label_file.name = os.path.basename(_LABEL_FILE)
label_file.description = "Labels for objects that the model can recognize."
label_file.type = _metadata_fb.AssociatedFileType.TENSOR_AXIS_LABELS
output_meta.associatedFiles = [label_file]

# Create subgraph metadata
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [output_meta]
model_meta.subgraphMetadata = [subgraph]

# Build the flatbuffer
b = flatbuffers.Builder(0)
b.Finish(
    model_meta.Pack(b),
    _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER
)
metadata_buf = b.Output()

# Populate metadata into the model
populator = _metadata.MetadataPopulator.with_model_file(_MODEL_PATH)
populator.load_metadata_buffer(metadata_buf)
populator.load_associated_files([_LABEL_FILE])
populator.populate()

# Save the model with metadata
writer_utils.save_file(writer_utils.load_file(_MODEL_PATH), _SAVE_TO_PATH)

# Optionally display metadata
displayer = _metadata.MetadataDisplayer.with_model_file(_SAVE_TO_PATH)
export_json_file = "model_metadata.json"
json_file = displayer.get_metadata_json()

with open(export_json_file, "w") as f:
    f.write(json_file)

print(f"Metadata populated and saved to {_SAVE_TO_PATH}")
print(f"Metadata JSON saved to {export_json_file}")


Metadata populated and saved to mobilenetv2_model_metadata.tflite
Metadata JSON saved to model_metadata.json


In [10]:
displayer = metadata.MetadataDisplayer.with_model_file("mobilenetv2_model_metadata.tflite")
print("Metadata populated:")
print(displayer.get_metadata_json())

print("Associated file(s) populated:")
for file_name in displayer.get_packed_associated_file_list():
  print("file name: ", file_name)
  print("file content:")
  print(displayer.get_associated_file_buffer(file_name))
with open("model_metadata.json", "w") as f:
  f.write(displayer.get_metadata_json())

Metadata populated:
{
  "name": "MobileNetV2 Gizilo image classifier",
  "description": "Identify branded retail products and make grading for these products based on the calculation of nutritional value.",
  "version": "v1",
  "subgraph_metadata": [
    {
      "input_tensor_metadata": [
        {
          "name": "image",
          "description": "Input image to be classified. The expected image is 229 x 229, with three channels (red, blue, and green) per pixel. Each value in the tensor is a single byte between 0 and 255.",
          "content": {
            "content_properties_type": "ImageProperties",
            "content_properties": {
              "color_space": "RGB"
            }
          },
          "process_units": [
            {
              "options_type": "NormalizationOptions",
              "options": {
                "mean": [
                  127.5
                ],
                "std": [
                  127.5
                ]
              }
            